# Installing unsloth

In [1]:
# mount google drive to load train dataset and save model...
from google.colab import drive
drive.mount('/content/drive')

drive_3gpp_experiment_path = r"/content/drive/MyDrive/doutorado/experimentos/3GPP"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Run these commands if you are using google colab.
%%capture
!pip3 install datasets
!pip3 install unsloth

# Also get the latest nightly Unsloth!
!pip3 uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!pip3 install transformers==4.49.0

# Running fine tuning

In [3]:
import torch
print("Torch Version: ", torch.__version__)
print("Torch Version Cuda: ", torch.version.cuda)
print("Torch Version cuDnn: ", torch.backends.cudnn.version())
print("CUDA Available: ", torch.cuda.is_available())
print("CUDA Device Name: ", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# check if CUDA is available to accelerate proccessing
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Torch Version:  2.6.0+cu124
Torch Version Cuda:  12.4
Torch Version cuDnn:  90100
CUDA Available:  True
CUDA Device Name:  Tesla T4
Using device: cuda


In [4]:
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

In [6]:
initial_model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name="unsloth/Llama-3.2-3B-bnb-4bit",
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-3B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # device_map="auto"
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = initial_model

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
# move model to CPU before delete
# model.to("cpu")
# delete reference to the model
# del model

# import gc and call garbage collect to free orphan objects
# import gc
# gc.collect()

# Free GPU memor
# torch.cuda.empty_cache()

In [7]:
initial_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )

# Load it from the disk

In [9]:
from datasets import load_from_disk

dataset_path = drive_3gpp_experiment_path + '/datasets/train/train_questions_dataset_4000_questions_short_answer_labels'

raw_dataset = load_from_disk(dataset_path)

print(len(raw_dataset))
print(raw_dataset)
print(raw_dataset[0])
print(raw_dataset[5])

4000
Dataset({
    features: ['conversations'],
    num_rows: 4000
})
{'conversations': [{'from': 'human', 'value': 'Question: What advantage does tUAV (tethered UAV) have in terms of backhaul communication compared to uUAVs (untethered UAVs)?\nOptions:\noption 1: tUAVs have a wireless backhaul link with higher capacity compared to uUAVs.\noption 2: tUAVs have more resource blocks for serving mobile users compared to uUAVs.\noption 3: tUAVs have a virtual tether that ensures strong and reliable backhaul link.\noption 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\noption 5: None of the above.\n'}, {'from': 'gpt', 'value': 'Answer: option 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\nExplanation: tUAVs have a stable wired connection to the GS with significantly higher capacity compared to a wireless backhaul link in uUAVs.'}]}
{'conversations': [{'from': 'human', 'value': 'Question: Which of the following deep lea

# Format TeleQnA Dataset for training

In [10]:
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_sharegpt

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

std_dataset = standardize_sharegpt(raw_dataset)
dataset = std_dataset.map(formatting_prompts_func, batched = True,)

print(dataset)

dataset[0]

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Dataset({
    features: ['conversations', 'text'],
    num_rows: 4000
})


{'conversations': [{'content': 'Question: What advantage does tUAV (tethered UAV) have in terms of backhaul communication compared to uUAVs (untethered UAVs)?\nOptions:\noption 1: tUAVs have a wireless backhaul link with higher capacity compared to uUAVs.\noption 2: tUAVs have more resource blocks for serving mobile users compared to uUAVs.\noption 3: tUAVs have a virtual tether that ensures strong and reliable backhaul link.\noption 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\noption 5: None of the above.\n',
   'role': 'user'},
  {'content': 'Answer: option 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\nExplanation: tUAVs have a stable wired connection to the GS with significantly higher capacity compared to a wireless backhaul link in uUAVs.',
   'role': 'assistant'}],
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<

# Training

In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
# if necessary run command below to delete model
# del model

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, DataCollatorWithPadding
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 300,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/4000 [00:00<?, ? examples/s]

In [13]:
trainer.train_dataset[0].items()

dict_items([('conversations', [{'content': 'Question: What advantage does tUAV (tethered UAV) have in terms of backhaul communication compared to uUAVs (untethered UAVs)?\nOptions:\noption 1: tUAVs have a wireless backhaul link with higher capacity compared to uUAVs.\noption 2: tUAVs have more resource blocks for serving mobile users compared to uUAVs.\noption 3: tUAVs have a virtual tether that ensures strong and reliable backhaul link.\noption 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\noption 5: None of the above.\n', 'role': 'user'}, {'content': 'Answer: option 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\nExplanation: tUAVs have a stable wired connection to the GS with significantly higher capacity compared to a wireless backhaul link in uUAVs.', 'role': 'assistant'}]), ('text', '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024

In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/4000 [00:00<?, ? examples/s]

In [15]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: What advantage does tUAV (tethered UAV) have in terms of backhaul communication compared to uUAVs (untethered UAVs)?\nOptions:\noption 1: tUAVs have a wireless backhaul link with higher capacity compared to uUAVs.\noption 2: tUAVs have more resource blocks for serving mobile users compared to uUAVs.\noption 3: tUAVs have a virtual tether that ensures strong and reliable backhaul link.\noption 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\noption 5: None of the above.\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAnswer: option 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\nExplanation: tUAVs have a stable wired connection to the GS with significantly higher capacity compared to a wireless ba

In [16]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]])

'                                                                                                                                                                              Answer: option 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\nExplanation: tUAVs have a stable wired connection to the GS with significantly higher capacity compared to a wireless backhaul link in uUAVs.<|eot_id|>'

# Show current memory stats

In [18]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.139 GB of memory reserved.


# Initial Loss

In [19]:
trainer.train_dataset[0].items()

dict_items([('conversations', [{'content': 'Question: What advantage does tUAV (tethered UAV) have in terms of backhaul communication compared to uUAVs (untethered UAVs)?\nOptions:\noption 1: tUAVs have a wireless backhaul link with higher capacity compared to uUAVs.\noption 2: tUAVs have more resource blocks for serving mobile users compared to uUAVs.\noption 3: tUAVs have a virtual tether that ensures strong and reliable backhaul link.\noption 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\noption 5: None of the above.\n', 'role': 'user'}, {'content': 'Answer: option 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\nExplanation: tUAVs have a stable wired connection to the GS with significantly higher capacity compared to a wireless backhaul link in uUAVs.', 'role': 'assistant'}]), ('text', '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024

In [20]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Configura o collator e DataLoader
collator = DataCollatorForSeq2Seq(tokenizer)
loader = DataLoader(trainer.train_dataset.remove_columns(["conversations", "text"]),
                    batch_size=2,  # Tamanho do batch escolhido
                    collate_fn=collator,
                    num_workers=2)

# Variáveis para armazenar a loss total e o número de exemplos
total_loss = 0.0
num_batches = 0

# Coloca o modelo em modo de avaliação
model.eval()

# Desativa o cálculo de gradiente para economizar memória
with torch.no_grad():
    for batch in tqdm(loader, desc="Calculating initial loss"):
        # Move o batch para a GPU (se disponível)
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Acumula a loss
        total_loss += outputs.loss.item()
        num_batches += 1

# Calcula a loss média
average_loss = total_loss / num_batches
print(f"Initial mean loss: {average_loss}")

Calculating initial loss: 100%|██████████| 2000/2000 [05:03<00:00,  6.60it/s]

Initial mean loss: 2.3857014101594687


# Training Session

In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,000 | Num Epochs = 3 | Total steps = 300
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 16 x 1) = 32
 "-____-"     Trainable parameters = 24,313,856/3,237,063,680 (0.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.387900
2,1.394500
3,1.898800
4,1.964200
5,1.288500
6,1.101800
7,1.363400
8,1.041000
9,0.867600
10,1.047700


# Show final memory and time stats

In [22]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

model

2021.9932 seconds used for training.
33.7 minutes used for training.
Peak reserved memory = 8.369 GB.
Peak reserved memory for training = 2.23 GB.
Peak reserved memory % of max memory = 56.774 %.
Peak reserved memory for training % of max memory = 15.128 %.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

# Loss Pos training

In [23]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Configure the collator and DataLoader
collator = DataCollatorForSeq2Seq(tokenizer)
loader = DataLoader(
    trainer.train_dataset.remove_columns(["conversations", "text"]),
    batch_size=2,  # Chosen batch size
    collate_fn=collator,
    num_workers=2
)

# Variables to store the total loss and the number of valid batches
total_loss = 0.0
num_batches = 0

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation to save memory
with torch.no_grad():
    for batch in tqdm(loader, desc="Calculating loss post-training"):
        # Move the batch to GPU (if available)
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Check if the loss is NaN
        if not torch.isnan(outputs.loss):
            total_loss += outputs.loss.item()
            num_batches += 1
            # print(outputs.loss.item())

print(f"Number of valid batches: {num_batches}")

# Calculate the average loss
if num_batches > 0:
    average_loss = total_loss / num_batches
    print(f"Post-training mean loss: {average_loss}")
else:
    print("No valid batches found.")

Calculating loss post-training: 100%|██████████| 2000/2000 [04:59<00:00,  6.67it/s]

Number of valid batches: 2000
Post-training mean loss: 0.7464656528108753


# Inference

## Question with option

In [26]:
print(dataset[5]['conversations'][0])
question = dataset[5]['conversations'][0]['content']

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


{'content': 'Question: Which of the following deep learning libraries provides a sophisticated visualization tool called TensorBoard?\nOptions:\noption 1: MXNET\noption 2: Theano\noption 3: Caffe\noption 4: PyTorch\noption 5: TensorFlow\n', 'role': 'user'}


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Which of the following deep learning libraries provides a sophisticated visualization tool called TensorBoard?\nOptions:\noption 1: MXNET\noption 2: Theano\noption 3: Caffe\noption 4: PyTorch\noption 5: TensorFlow\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAnswer: option 5: TensorFlow\nExplanation: TensorFlow provides a sophisticated visualization tool called TensorBoard.<|eot_id|>']

## Question with no option

In [27]:
print(dataset[0]['conversations'][0]['content'])
question = dataset[0]['conversations'][0]['content']

from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

Question: What advantage does tUAV (tethered UAV) have in terms of backhaul communication compared to uUAVs (untethered UAVs)?
Options:
option 1: tUAVs have a wireless backhaul link with higher capacity compared to uUAVs.
option 2: tUAVs have more resource blocks for serving mobile users compared to uUAVs.
option 3: tUAVs have a virtual tether that ensures strong and reliable backhaul link.
option 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.
option 5: None of the above.



['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: What advantage does tUAV (tethered UAV) have in terms of backhaul communication compared to uUAVs (untethered UAVs)?\nOptions:\noption 1: tUAVs have a wireless backhaul link with higher capacity compared to uUAVs.\noption 2: tUAVs have more resource blocks for serving mobile users compared to uUAVs.\noption 3: tUAVs have a virtual tether that ensures strong and reliable backhaul link.\noption 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\noption 5: None of the above.\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAnswer: option 4: tUAVs have a wired connection to the GS with higher capacity compared to uUAVs.\nExplanation: Compared to uUAVs, tUAVs have a wired backhaul connection with significantly higher capacity due to direct connection to th

# Save model

In [28]:
model.save_pretrained(drive_3gpp_experiment_path + "/models/llama_3.2_1B_FT_lora_4000_questions_short_answer_labels")
tokenizer.save_pretrained(drive_3gpp_experiment_path + "/models/llama_3.2_1B_FT_lora_4000_questions_short_answer_labels")

('/content/drive/MyDrive/doutorado/experimentos/3GPP/models/llama_3.2_1B_FT_lora_4000_questions_short_answer_labels/tokenizer_config.json',
 '/content/drive/MyDrive/doutorado/experimentos/3GPP/models/llama_3.2_1B_FT_lora_4000_questions_short_answer_labels/special_tokens_map.json',
 '/content/drive/MyDrive/doutorado/experimentos/3GPP/models/llama_3.2_1B_FT_lora_4000_questions_short_answer_labels/tokenizer.json')

## Save merged

In [ ]:
# model.save_pretrained_merged("../models/llama_3.2_1B_FT_lora_4bits_4000_questions", tokenizer, save_method = "merged_4bit",)

# Load Model

In [29]:
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [30]:
model_path = drive_3gpp_experiment_path + "/models/llama_3.2_1B_FT_lora_4000_questions_short_answer_labels"
# model_path = "model_3.2_lora_4bits"

# Carregar o modelo e o tokenizador separadamente
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype = dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.49.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

In [31]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear